In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Venks\\Desktop\\Project\\MLOPS_Bootcamp\\Kidney_Classification\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Venks\\Desktop\\Project\\MLOPS_Bootcamp\\Kidney_Classification'

Entity

import dagshub
dagshub.init(repo_owner='iamvenkatesh14', repo_name='Kidney_Disease_Image_Classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

  https://dagshub.com/user/settings/tokens - Default access token: 02f9de3e0f346e13677dcb32372f441d7b90d3e6
  export MLFLOW_TRACKING_URI=https://dagshub.com/iamvenkatesh14/Kidney_Disease_Image_Classification.mlflow
  export MLFLOW_TRACKING_USERNAME=iamvenkatesh14
  export MLFLOW_TRACKING_PASSWORD=02f9de3e0f346e13677dcb32372f441d7b90d3e6

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/iamvenkatesh14/Kidney_Disease_Image_Classification.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="iamvenkatesh14"
os.environ["MLFLOW_TRACKING_PASSWORD"]="02f9de3e0f346e13677dcb32372f441d7b90d3e6"

In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:
from Kidney_Disease.constants import *
from Kidney_Disease.utils.common import read_yaml, create_directories, save_json

Config folder

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CT-Kidney-Disease-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/iamvenkatesh14/Kidney_Disease_Image_Classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




Components: -> mlflow_model_eval.py

In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the modelscikit learn will be used for ML projects eg: mlflow.scikit-learn
                #Keras will be used for deeplearning and 
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-11-25 20:33:16,013: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-25 20:33:16,019: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-25 20:33:16,020: INFO: common: created directory at: artifacts]
Found 3732 images belonging to 4 classes.
234/234 [==============================] - 563s 2s/step - loss: 5.2740 - accuracy: 0.4920
[2024-11-25 20:42:40,300: INFO: common: json file saved at: scores.json]


2024/11/25 20:42:43 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-11-25 20:42:44,650: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
[2024-11-25 20:42:45,354: INFO: builder_impl: Assets written to: C:\Users\Venks\AppData\Local\Temp\tmpbuzz2_cl\model\data\model\assets]


c:\Users\Venks\Desktop\Project\MLOPS_Bootcamp\Kidney_Classification\kidny_env_3.10\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2024/11/25 20:43:25 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
